In [1]:
import numpy as np
from flask import Flask, request, jsonify
import pickle
import pandas as pd
from newspaper import Article
from textblob import Word, TextBlob
from nltk.corpus import stopwords
from flask_cors import CORS



In [2]:
# Load the COunt Vectoriser
count_vect = pickle.load(open("count_vect.p","rb"))

# Load the model
nb = pickle.load(open('nb.p','rb'))
rf = pickle.load(open('rf.p','rb'))
logreg = pickle.load(open('logreg.p','rb'))
svm = pickle.load(open('svm.p','rb'))

In [3]:
app = Flask(__name__)
CORS(app)

In [4]:
@app.route('/',methods=['POST'])
def predict():
    # Get the data from the POST request.
    data = request.get_json(force=True)   
    Url = data['url']
    
    test_article = Article(Url, language="en")
    
    #To download the article 
    test_article.download() 
    
    #To parse the article 
    test_article.parse() 
    
    #To perform natural language processing ie..nlp 
    test_article.nlp() 
    
    # Extracting news article
    
    sampleArticle = TextBlob(test_article.text)
    sample = pd.DataFrame([test_article.text])
    
    # Sentiment Analysis
    
    PI = sampleArticle.polarity
    SI = sampleArticle.subjectivity
    
    # Doing some preprocessing on these samples as done before
    sample[0] = sample[0].str.replace('[^\w\s]',' ')
    stop = stopwords.words('english')
    sample[0] = sample[0].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    sample[0] = sample[0].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    
    # Extracting Count Vectors feature from our news article
    sample_count = count_vect.transform(sample[0])
    
    RI = 0
    
    #real = 1, fake = 0
    
    if (nb.predict(sample_count) == 1):
        RI = RI + 25
    if (svm.predict(sample_count) == 1):
        RI = RI + 25
    if (rf.predict(sample_count) == 1):
        RI = RI + 25
    if (logreg.predict(sample_count) == 1):
        RI = RI + 25
    
    return jsonify({'RI' : str(RI), "PI" : str(PI), "SI" : str(SI)})

In [ ]:
if __name__ == '__main__':
    app.run(port=5000, debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2019 09:34:21] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:34:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:35:22] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 09:38:44] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 11:23:25] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 12:39:37] "POST / HTTP/1.1" 200 -
[2019-04-11 13:36:13,995] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\JAS\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\JAS\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\JAS\Anaconda3\lib\site-packages\flask_cors\extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\JAS\Anaconda3\li